In [ ]:

import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'tomatopy'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/tomatopy/2017.csv")

In [ ]:
print(df.head())

   Unnamed: 0  Rank      Release Group     Worldwide      Domestic      %  \
0           0     1         The Batman  $603,263,177  $302,863,177  50.2%   
1           1     2  Water Gate Bridge  $480,203,271      $117,294  <0.1%   
2           2     3          Uncharted  $337,762,979  $126,362,979  37.4%   
3           3     4   Too Cool to Kill  $217,255,728      $185,882  <0.1%   
4           4     5             Scream  $139,813,281   $81,413,281  58.2%   

        Foreign    %.1  
0  $300,400,000  49.8%  
1  $480,085,977   100%  
2  $211,400,000  62.6%  
3  $217,069,846  99.9%  
4   $58,400,000  41.8%  


In [ ]:
df1 = pd.DataFrame([],columns = ['Release Group', 'Score_Rotten','Score_Audience','Rating','Genre','Director','Producer','Release Date (Theaters)','Box Office (Gross USA)'])

In [ ]:
df1['Release Group'] = df['Release Group']

In [ ]:
from urllib.request import urlopen

In [ ]:
score_rotten = []
score_Audience = []
rating = []
genre = []
director = []
producer = []
release_Date = []
box_Office = []
for movie in df1['Release Group']:
  
  movie_scraper = MovieScraper(movie_title=movie)
  try:
    page_movie = urlopen(movie_scraper.url)
  except:
    movie_scraper.url = None
  if movie_scraper.url is not None:
    print(movie_scraper.url)

    movie_scraper.extract_metadata()
    score_rotten.append(movie_scraper.metadata['Score_Rotten'])
    score_Audience.append(movie_scraper.metadata['Score_Audience'])
    rating.append(movie_scraper.metadata['Rating'])
    genre.append(movie_scraper.metadata['Genre'])
    director.append(movie_scraper.metadata['Director'])
    producer.append(movie_scraper.metadata['Producer'])
    release_Date.append(movie_scraper.metadata['Release Date (Theaters)'])
    box_Office.append(movie_scraper.metadata['Box Office (Gross USA)'])
  else:
    score_rotten.append(None)
    score_Audience.append(None)
    genre.append(None)
    rating.append(None)
    director.append(None)
    producer.append(None)
    release_Date.append(None)
    box_Office.append(None)
df1['Score_Rotten'] = score_rotten
df1['Score_Audience'] = score_Audience
df1['Genre'] = genre
df1['Rating'] = rating
df1['Director'] = director
df1['Producer'] = producer
df1['Release Date (Theaters)'] = release_Date
df1['Box Office (Gross USA)'] = box_Office
df1.to_csv("/content/gdrive/MyDrive/tomatopy/new-rotten/2017_info.csv", encoding='utf-8')

https://www.rottentomatoes.com/m/beauty_and_the_beast_2017
https://www.rottentomatoes.com/m/the_fate_of_the_furious
https://www.rottentomatoes.com/m/despicable_me_3
https://www.rottentomatoes.com/m/jumanji_welcome_to_the_jungle
https://www.rottentomatoes.com/m/spider_man_homecoming
https://www.rottentomatoes.com/m/wolf_warrior_ii
https://www.rottentomatoes.com/m/guardians_of_the_galaxy_vol_2
https://www.rottentomatoes.com/m/thor_ragnarok_2017
https://www.rottentomatoes.com/m/wonder_woman_2017
https://www.rottentomatoes.com/m/coco_2017
https://www.rottentomatoes.com/m/pirates_of_the_caribbean_dead_men_tell_no_tales
https://www.rottentomatoes.com/m/justice_league_2017
https://www.rottentomatoes.com/m/logan_2017
https://www.rottentomatoes.com/m/transformers_the_last_knight_2017
https://www.rottentomatoes.com/m/kong_skull_island
https://www.rottentomatoes.com/m/the_boss_baby
https://www.rottentomatoes.com/m/dunkirk_2017
https://www.rottentomatoes.com/m/war_for_the_planet_of_the_apes
https:

In [ ]:
from bs4 import BeautifulSoup
import difflib
import re
import requests
from urllib.request import urlopen
from collections import defaultdict


class RTScraper:
    BASE_URL = "https://www.rottentomatoes.com/api/private/v2.0"
    SEARCH_URL = "{base_url}/search".format(base_url=BASE_URL)

    def __init__(self):
        self.metadata = dict()
        self.url = None

    def extract_url(self):
        pass

    def extract_metadata(self, **kwargs):
        pass

    def _extract_section(self, section):
        pass

    @staticmethod
    def search(term, limit=10):
        r = requests.get(url=RTScraper.SEARCH_URL, params={"q": term, "limit": limit})
        r.raise_for_status()
        return r.json()


class MovieScraper(RTScraper):
    def __init__(self, **kwargs):
        RTScraper.__init__(self)
        self.movie_genre = None
        if 'movie_title' in kwargs.keys():
            self.movie_title = kwargs['movie_title']
            self.extract_url()
        if 'movie_url' in kwargs.keys():
            self.url = kwargs['movie_url']

    def extract_url(self):
        search_result = self.search(term=self.movie_title)
        movie_titles = []
        for movie in search_result['movies']:
            movie_titles.append(movie['name'])

        closest = self.closest(self.movie_title, movie_titles)
        url_movie = None
        if len(closest)>0:
          for movie in search_result['movies']:
              if movie['name'] == closest[0]:
                  url_movie = 'https://www.rottentomatoes.com' + movie['url']

        self.url = url_movie

    def extract_metadata(self, columns=('Rating', 'Genre', 'Box Office', 'Studio','Director','Producer','Release Date (Theaters)','Box Office (Gross USA)')):
        movie_metadata = dict()
        page_movie = urlopen(self.url)
        soup = BeautifulSoup(page_movie, "lxml")

        # Score
        score = soup.find('score-board')
        movie_metadata['Score_Rotten'] = score.attrs['tomatometerscore']
        movie_metadata['Score_Audience'] = score.attrs['audiencescore']

        # Movie Info
        movie_info_section = soup.find_all('div', class_='media-body')
        soup_movie_info = BeautifulSoup(str(movie_info_section[0]), "lxml")
        movie_info_length = len(soup_movie_info.find_all('li', class_='meta-row clearfix'))

        import re

        monthsShort="Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec"
        monthsLong="January|February|March|April|May|June|July|August|September|October|November|December"
        months="(" + monthsShort + "|" + monthsLong + ")"

        days = "\d{2}"
        years = "\d{4}"

        regex1 = months +" "+ days

        for i in range(movie_info_length):
            x = soup_movie_info.find_all('li', class_='meta-row clearfix')[i]
            soup = BeautifulSoup(str(x), "lxml")
            label = soup.find('div', class_='meta-label subtle').text.strip().replace(':', '')
            value = soup.find('div', class_='meta-value').text.strip()
            if label in columns:
                if label == 'Box Office':
                    value = int(value.replace('$', '').replace(',', ''))
                if label == 'Rating':
                    value = re.sub(r'\s\([^)]*\)', '', value)
                if label == 'Genre':
                    value = value.replace('\n', '').split(',')
                    remove_space = [x.strip(' ') for x in value]
                    value = remove_space
                if label == 'Director':
                    value = value.replace('\n', '').split(',')
                    remove_space = [x.strip(' ') for x in value]
                    value = remove_space
                if label == 'Producer':
                    value = value.replace('\n', '').split(',')
                    remove_space = [x.strip(' ') for x in value]
                    value = remove_space
                if label == 'Release Date (Theaters)':
                    l = value.split(',')
                    l1 = l[1].split()
                    res = []
                    res.append(l[0])
                    res.append(l1[0])
                    value = res
                if label == 'Box Office (Gross USA)':
                    value = value.replace('$', '').replace(',', '')
                movie_metadata[label] = value
        keys = movie_metadata.keys()
        for label in columns: 
          if label not in keys:
            movie_metadata[label] = None


        self.metadata = movie_metadata
        self.movie_genre = self.extract_genre(self.metadata)

    @staticmethod
    def closest(keyword, words):
        closest_match = difflib.get_close_matches(keyword, words, cutoff=0.6)
        return closest_match

    @staticmethod
    def extract_genre(metadata):
        try:
            if 'Genre' in metadata:
                movie_genre = metadata['Genre']
            else:
                movie_genre = ['None']

        except IOError:
            movie_genre = ['None']

        return movie_genre


class CelebrityScraper(RTScraper):
    def __init__(self, **kwargs):
        RTScraper.__init__(self)
        if 'celebrity_name' in kwargs.keys():
            self.celebrity_name = kwargs['celebrity_name']
            self.extract_url()
        if 'celebrity_url' in kwargs.keys():
            self.url = kwargs['celebrity_url']

    def extract_url(self):
        search_result = self.search(term=self.celebrity_name)
        url_celebrity = 'https://www.rottentomatoes.com' + search_result['actors'][0]['url']
        self.url = url_celebrity

    def _extract_section(self, section):
        page_celebrity = urlopen(self.url)
        soup = BeautifulSoup(page_celebrity, "lxml")
        selected_section = []
        try:
            if section == 'highest':
                selected_section = soup.find_all('section', class_='dynamic-poster-list')[0].text.split('\n')
            elif section == 'filmography':
                selected_section = soup.find_all('tbody', class_='celebrity-filmography__tbody')[0]
        except IOError:
            print('The parsing process returns an error.')

        return selected_section

    def extract_metadata(self, section):
        selected_section = self._extract_section(section=section)
        movie_titles = []
        if section == 'highest':
            for i in range(len(selected_section)):
                if selected_section[i].strip():
                    movie_titles.append(selected_section[i].strip())
            movie_titles.remove('Highest rated movies')
        elif section == 'filmography':
            soup_filmography = BeautifulSoup(str(selected_section), 'lxml')
            for h in soup_filmography.find_all('a'):
                try:
                    movie_titles.append(h.text.strip())
                except IOError:
                    pass

        self.metadata['movie_titles'] = list(set(movie_titles))
    
    
class DirectorScraper(RTScraper):
    def __init__(self, **kwargs):
        RTScraper.__init__(self)
        if 'director_name' in kwargs.keys():
            self.director_name = kwargs['director_name']
            self.extract_url()
        if 'director_url' in kwargs.keys():
            self.url = kwargs['director_url']
        if 'print' in kwargs.keys():
            self.print = kwargs['print']

    def extract_url(self):
        search_result = self.search(term=self.director_name)
        url_director = 'https://www.rottentomatoes.com' + search_result['actors'][0]['url']
        self.url = url_director
    
    def extract_metadata(self):
        try:
            if self.print:
                try:
                    print(self.director_name, self.url)
                except AttributeError:
                    print(self.url)
        except AttributeError:
            pass
        page_director = urlopen(self.url)
        soup = BeautifulSoup(page_director, 'lxml')
        try:
            selected_section = soup.find_all('tbody', class_='celebrity-filmography__tbody')[0]
        except IOError:
            print('The parsing process returns an error.')

        soup_filmography = BeautifulSoup(str(selected_section), 'lxml')
        movie_metadata = defaultdict(dict)
        for each_row in soup_filmography.find_all('tr'):
            is_this_a_linked_movie = each_row.find('td', class_='celebrity-filmography__title').find('a')
            if is_this_a_linked_movie is None:
                next
            else:
                for each_cell in each_row.find_all('td', class_="celebrity-filmography__credits"):
                    for each_string in each_cell.stripped_strings:
                        if "Director" in each_string:
                            try:
                                this_title = each_row['data-title']
                                movie_metadata[this_title]['Year'] = each_row['data-year']
                                movie_metadata[this_title]['Score_Rotten'] = each_row['data-tomatometer']
                                movie_metadata[this_title]['Box_Office'] = each_row['data-boxoffice']
                            except IOError:
                                pass
        self.metadata = movie_metadata